In [ ]:
%pip install torch

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader

In [2]:
# =========================
# 0. Config de base
# =========================

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)

batch_size = 128
test_batch_size = 1000
lr = 1e-3

epochs_clean = 3      # nb d epoques pour le modele base
epochs_adv = 3        # nb d epoques pour le modele adversarial

epsilon = 0.25        # intensite de la perturbation FGSM
alpha_adv = 0.5       # poids clean vs adversarial dans la loss

Using device: cpu


In [4]:
# =========================
# 1. Chargement de MNIST
# =========================

transform = transforms.ToTensor()

train_dataset = datasets.MNIST(
    root="./data",
    train=True,
    download=True,
    transform=transform,
)

test_dataset = datasets.MNIST(
    root="./data",
    train=False,
    download=True,
    transform=transform,
)

train_loader= DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
train_loader_no_shuffle = DataLoader(train_dataset, batch_size=batch_size, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=test_batch_size, shuffle=False)


def describe_dataset(train_dataset, test_dataset):
    print("Taille train :", len(train_dataset))
    print("Taille test  :", len(test_dataset))
    print("Nombre de classes :", len(train_dataset.classes))
    print("Classes :", train_dataset.classes)

describe_dataset(train_dataset, test_dataset)

Taille train : 60000
Taille test  : 10000
Nombre de classes : 10
Classes : ['0 - zero', '1 - one', '2 - two', '3 - three', '4 - four', '5 - five', '6 - six', '7 - seven', '8 - eight', '9 - nine']


In [6]:
# =========================
# 2. Modele simple
# =========================

class SimpleCNN(nn.Module):
    def __init__(self):
        super().__init__()
        # entree: 1 x 28 x 28
        self.conv1 = nn.Conv2d(1, 32, kernel_size=3, padding=1)   # 32 x 28 x 28
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, padding=1)  # 64 x 28 x 28
        self.pool = nn.MaxPool2d(2, 2)                            # 64 x 14 x 14
        self.fc1 = nn.Linear(64 * 14 * 14, 128)
        self.fc2 = nn.Linear(128, 10)

    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = F.relu(self.conv2(x))
        x = self.pool(x)
        x = x.view(x.size(0), -1)  # flatten
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x

In [7]:
# =========================
# 3. Entrainement et test clean
# =========================

def train_one_epoch_clean(model, loader, optimizer, epoch, device):
    model.train()
    total_loss = 0
    correct = 0
    total = 0

    for data, target in loader:
        data, target = data.to(device), target.to(device)

        optimizer.zero_grad()
        output = model(data)
        loss = F.cross_entropy(output, target)
        loss.backward()
        optimizer.step()

        total_loss += loss.item() * data.size(0)
        pred = output.argmax(dim=1)
        correct += pred.eq(target).sum().item()
        total += data.size(0)

    avg_loss = total_loss / total
    acc = correct / total
    print(f"[Train clean] Epoch {epoch} - Loss: {avg_loss:.4f}, Acc: {acc:.4f}")


def test_model_clean(model, loader, device, label="Model"):
    model.eval()
    test_loss = 0
    correct = 0
    total = 0

    with torch.no_grad():
        for data, target in loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            loss = F.cross_entropy(output, target, reduction="sum")
            test_loss += loss.item()
            pred = output.argmax(dim=1)
            correct += pred.eq(target).sum().item()
            total += data.size(0)

    avg_loss = test_loss / total
    acc = correct / total
    print(f"[{label}] Test clean - Loss: {avg_loss:.4f}, Acc: {acc:.4f}")
    return avg_loss, acc

In [8]:
# =========================
# 4. FGSM - Fast Gradient Sign Method
# =========================

def fgsm_attack(model, images, labels, epsilon, device):
    """
    FGSM: x_adv = x + epsilon * sign(grad_x J(theta, x, y))
    """
    model.eval()

    images = images.clone().detach().to(device)
    images.requires_grad = True
    labels = labels.to(device)

    outputs = model(images)
    loss = F.cross_entropy(outputs, labels)

    model.zero_grad()
    loss.backward()

    grad_sign = images.grad.data.sign()
    adv_images = images + epsilon * grad_sign
    adv_images = torch.clamp(adv_images, 0, 1)

    return adv_images.detach()


In [9]:
# =========================
# 5. Test sur adversarial examples
# =========================

def test_model_adversarial(model, loader, epsilon, device, label="Model"):
    model.eval()
    adv_loss = 0
    correct = 0
    total = 0

    for data, target in loader:
        data, target = data.to(device), target.to(device)

        adv_data = fgsm_attack(model, data, target, epsilon, device)

        outputs = model(adv_data)
        loss = F.cross_entropy(outputs, target, reduction="sum")
        adv_loss += loss.item()

        pred = outputs.argmax(dim=1)
        correct += pred.eq(target).sum().item()
        total += data.size(0)

    avg_loss = adv_loss / total
    acc = correct / total
    print(f"[{label}] Test adversarial eps={epsilon} - Loss: {avg_loss:.4f}, Acc: {acc:.4f}")
    return avg_loss, acc

In [10]:
# =========================
# 6. Main
# =========================

def main():
    # ----- Modele de base-----
    print("\n=== Entrainement du modele de base (clean) ===")
    torch.manual_seed(1)
    base_model = SimpleCNN().to(device)
    optimizer_base = optim.Adam(base_model.parameters(), lr=lr)

    for epoch in range(1, epochs_clean + 1):
        train_one_epoch_clean(base_model, train_loader, optimizer_base, epoch, device)
        test_model_clean(base_model, test_loader, device, label="Base model")

    print(f"\nEvaluation du modele : de base sur adversarial examples:")
    base_clean_loss, base_clean_acc = test_model_clean(base_model, test_loader, device, label="Base model")
    base_adv_loss, base_adv_acc = test_model_adversarial(base_model, test_loader, epsilon, device, label="Base model")

# ----- Modele initialisé différement-----
    print("\n=== Entrainement de la seconde initialisation ===")
    torch.manual_seed(2)
    base_model_2 = SimpleCNN().to(device)
    optimizer_base_2 = optim.Adam(base_model_2.parameters(), lr=lr)

    for epoch in range(1, epochs_clean + 1):
        train_one_epoch_clean(base_model_2, train_loader, optimizer_base_2, epoch, device)
        test_model_clean(base_model_2, test_loader, device, label="Base model 2")

    print(f"\nEvaluation du modele initialisé différement (poids) sur adversarial examples:")
    base_clean_loss_2, base_clean_acc_2 = test_model_clean(base_model_2, test_loader, device, label="Base model 2")
    base_adv_loss_2, base_adv_acc_2 = test_model_adversarial(base_model_2, test_loader, epsilon, device, label="Base model 2")

# ----- Modele entrainé sans shuffle-----
    print("\n=== Entrainement de la seconde initialisation ===")
    torch.manual_seed(1)
    base_model_3 = SimpleCNN().to(device)
    optimizer_base_3 = optim.Adam(base_model_3.parameters(), lr=lr)

    for epoch in range(1, epochs_clean + 1):
        train_one_epoch_clean(base_model_3, train_loader_no_shuffle, optimizer_base_3, epoch, device)
        test_model_clean(base_model_3, test_loader, device, label="Base model 3")

    print(f"\nEvaluation du modele entrainé sans shuffle sur adversarial examples:")
    base_clean_loss_3, base_clean_acc_3 = test_model_clean(base_model_3, test_loader, device, label="Base model 3")
    base_adv_loss_3, base_adv_acc_3 = test_model_adversarial(base_model_3, test_loader, epsilon, device, label="Base model 3")

    print("\n=== Resume comparatif ===")
    print(f"Modele de base - Clean Acc: {base_clean_acc:.4f}, Adv Acc: {base_adv_acc:.4f}")
    print(f"Modele avec poids différents - Clean ACC: {base_clean_acc_2:.4f}, Adv Acc: {base_adv_acc:.4f}")
    print(f"Modele entrainé sans shuffle - Clean ACC: {base_clean_acc_3:.4f}, Adv Acc: {base_adv_acc:.4F}")

if __name__ == "__main__":
    main()


=== Entrainement du modele de base (clean) ===
[Train clean] Epoch 1 - Loss: 0.1953, Acc: 0.9414
[Base model] Test clean - Loss: 0.0584, Acc: 0.9817
[Train clean] Epoch 2 - Loss: 0.0501, Acc: 0.9840
[Base model] Test clean - Loss: 0.0660, Acc: 0.9775
[Train clean] Epoch 3 - Loss: 0.0324, Acc: 0.9893
[Base model] Test clean - Loss: 0.0335, Acc: 0.9892

Evaluation du modele : de base sur adversarial examples:
[Base model] Test clean - Loss: 0.0335, Acc: 0.9892
[Base model] Test adversarial eps=0.25 - Loss: 3.9695, Acc: 0.1309

=== Entrainement de la seconde initialisation ===
[Train clean] Epoch 1 - Loss: 0.1976, Acc: 0.9407
[Base model 2] Test clean - Loss: 0.0577, Acc: 0.9820
[Train clean] Epoch 2 - Loss: 0.0491, Acc: 0.9849
[Base model 2] Test clean - Loss: 0.0441, Acc: 0.9860
[Train clean] Epoch 3 - Loss: 0.0325, Acc: 0.9896
[Base model 2] Test clean - Loss: 0.0405, Acc: 0.9867

Evaluation du modele initialisé différement (poids) sur adversarial examples:
[Base model 2] Test clean -